# OpenAI Responses API

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing a web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API

In [1]:
pip install openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

In [4]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [5]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_691264901b80819190cbf0e56690d39d


### Upload Files

In [6]:
with open('tweet_text.json', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-3PbL26Rbs32pogXwnwCrZw


### Attach File to Vector Store

In [7]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-3PbL26Rbs32pogXwnwCrZw


### Query the Vector Store

In [8]:
query = "the latest development in generativeAI"

In [9]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

## OpenAI Response API

### Simple Response

In [10]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [11]:
display(Markdown(simple_response.output_text))

As of the latest updates in 2023, several key developments have emerged in the field of generative AI:

1. **Advanced Multimodal Models**: New models can handle and integrate multiple types of data such as text, images, audio, and video, increasing their versatility and application range.

2. **Improved Scaling and Efficiency**: Efforts to make large language models more efficient and environmentally friendly are underway, focusing on reducing computational costs and energy consumption.

3. **Fine-Tuning and Customization**: Increased emphasis on fine-tuning models for specific applications or industries, allowing for more targeted and effective outputs.

4. **Ethical AI and Bias Reduction**: Ongoing work to address ethical concerns, bias mitigation, and the development of guidelines to ensure responsible AI deployment.

5. **Real-Time and Interactive Applications**: Enhanced capabilities for real-time interaction in applications like virtual assistants, chatbots, and collaborative creative tools.

6. **Creative and Artistic Collaborations**: Generative AI is inspiring new forms of art and content creation, being used in music, visual arts, and storytelling.

7. **Increased Accessibility**: More user-friendly platforms and tools are being developed to make generative AI accessible to non-experts, expanding its use across various sectors.

These trends highlight the rapid advancement and integration of generative AI into everyday applications and industries.

### File Search Response

In [12]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [13]:
display(Markdown(file_search_response.output_text))


The latest developments in generative AI include several exciting advancements:

1. **OpenAI's Sora2**: This tool allows for the creation of cinematic videos from a simple prompt, incorporating audio, physics, and cameos, which is a game-changer for creators.

2. **Generative AI in Business**: Companies are using generative AI to innovate across various sectors, from content creation to strategic planning.

3. **AI in Supply Chain**: Atos has developed an AI-powered Supply Chain Disruption Analysis using generative AI to assess risks and boost resilience.

4. **Enterprise Applications**: IBM's Watsonx is bringing generative AI to enterprises, allowing teams to build custom large language models to enhance customer engagement and streamline processes.

5. **Creative Industries**: Generative AI is transforming creative industries by enabling new forms of content creation and design.

These developments highlight the diverse applications and transformative potential of generative AI across different fields.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [14]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [15]:
display(Markdown(web_search_response.output_text))

Here is a structured and in‑depth overview of the **latest developments in generative AI** as of Monday, November 10, 2025. This summary spans key model advancements, new tools and infrastructure, strategic industry moves, and emerging research directions.

---

##  Generative AI Model Milestones

- **OpenAI’s GPT‑5 (released August 7, 2025)**  
  GPT‑5 represents a major leap in multimodal generative AI, integrating reasoning, fast and high-throughput models, and autonomous tool use under one system. The architecture includes multiple specialized variants—“main”, “thinking”, and “nano” for optimized performance, with dynamic routing to select the appropriate model per task. Access is available via ChatGPT, Microsoft Copilot, and the OpenAI API.  
  ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai))

- **Google’s Gemini 2.5 Pro and Nano Banana**  
  Google upgraded its flagship model with the release of Gemini 2.5 Pro, offering enhanced reasoning, coding, and multimodal capabilities, including audio output and a “Deep Think” mode for complex tasks.  
  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gemini_%28language_model%29?utm_source=openai))  
  In August 2025, Google DeepMind launched “Nano Banana” (officially Gemini 2.5 Flash Image), a viral text-to-image generation tool capable of photorealistic “3D figurine” edits, with features like subject consistency, multi-image fusion, context awareness, and invisible SynthID watermarking.  
  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai))

- **Runway’s Gen‑4 for Text-to-Video Creation**  
  Released March 31, 2025, Gen‑4 is a text-to-video model that produces short (5–10 seconds), 720p, 24 fps video clips with consistent character behavior and simulated camera motion using diffusion-based architectures.  
  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gen-4_%28AI_image_and_video_model%29?utm_source=openai))

- **OpenAI’s Sora and Other Emerging Capabilities**  
  While not yet widely publicized, OpenAI’s Sora—an AI capable of generating videos from text prompts—received positive early buzz for its realism and speed, although full confirmation of its release status remains unclear.  
  ([nexthorizon.space](https://www.nexthorizon.space/2025/04/generative-ai-breakthroughs-2025-whats-new-and-what-truly-impressed.html?utm_source=openai))

- **Open‑Source Model Advances: Alibaba Qwen and Meta Llama 4**  
  Alibaba has significantly expanded its Qwen model family—including the multimodal Qwen2.5‑Omni and large-scale Qwen3 series—some components released under Apache 2.0 license, with “thinking” capabilities enabled for Qwen3‑Max in early November 2025.  
  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Qwen?utm_source=openai))  
  Meta introduced its Llama 4 family in April 2025, featuring mixture-of-experts (MoE) models like Scout (109B parameters) and Maverick (402B parameters), supporting massive 10M-token context windows and multimodal input, enabling advanced scalability with efficiency.  
  ([ts2.tech](https://ts2.tech/en/generative-ai-revolution-2025-breakthroughs-industry-disruption-and-predictions-through-2035/?utm_source=openai))

---

##  Hardware, Infrastructure, and Ecosystem Trends

- **Qualcomm’s AI200 and AI250 Accelerators (arriving 2026–2027)**  
  Qualcomm announced the AI200 and AI250 rack-scale inference accelerators, featuring advanced Hexagon NPUs with micro-tile inferencing, 64-bit addressing, model encryption, and massive LPDDR memory. These target large-scale generative AI workloads with improved efficiency and integration into major AI frameworks.  
  ([tomshardware.com](https://www.tomshardware.com/tech-industry/artificial-intelligence/qualcomm-unveils-ai200-and-ai250-ai-inference-accelerators-hexagon-takes-on-amd-and-nvidia-in-the-booming-data-center-realm?utm_source=openai))

- **Nvidia’s Next-Generation AI Chips**  
  At GTC 2025, Nvidia revealed its forthcoming Blackwell Ultra (second half of 2025), Vera Rubin (late 2026), and Rubin Ultra (2027) architectures—aimed at powering reasoning- and agent-capable generative AI workloads. Nvidia also unveiled its Isaac GR00T N1 humanoid robot model, the Cosmos synthetic data platform for robotics, and the Newton physics engine (developed with Google DeepMind and Disney Research) for robotics simulation.  
  ([apnews.com](https://apnews.com/article/457e9260aa2a34c1bbcc07c98b7a0555?utm_source=openai))

- **OpenAI–Oracle Stargate Data Center Expansion**  
  OpenAI, in partnership with Oracle (and supported by SoftBank), is expanding its Stargate AI infrastructure initiative with an additional 4.5 GW of computing capacity—bringing total deployment above 5 GW across more than 2 million chips. The broader plan targets up to 10 GW and a potential $500 billion investment.  
  ([reuters.com](https://www.reuters.com/business/openai-oracle-deepen-ai-data-center-push-with-45-gigawatt-stargate-expansion-2025-07-22/?utm_source=openai))

- **AWS GAIA Accelerator Program**  
  Amazon Web Services has launched its third Generative AI Accelerator (GAIA) cohort of 40 startups worldwide, offering eight weeks of mentoring and tools to help scale foundational model and infrastructure innovation.  
  ([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/technology/tech-news/aws-selects-40-startups-for-2025-generative-ai-accelerator-program/articleshow/124377434.cms?utm_source=openai))

---

##  Business, Media & Creative Applications

- **TIME AI Agent—A New Frontier in Interactive Journalism**  
  On November 10, 2025 (today), TIME launched the TIME AI Agent, a unified generative AI platform developed with Scale AI. It enables personalized summaries, audio reports, translations, and interactive content powered by robust moderation and data governance—marking a shift toward truly interactive AI-driven journalism.  
  ([time.com](https://time.com/7332572/the-story-behind-the-time-ai-agent/?utm_source=openai))

- **AI‑Driven Filmmaking with Darren Aronofsky’s Primordial Soup**  
  Filmmaker Darren Aronofsky’s creative studio—Primordial Soup—partnered with Google DeepMind to merge live-action with generative visuals in the short film *ANCESTRA*, slated for debut at the 2025 Tribeca Festival.  
  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Primordial_Soup_%28studio%29?utm_source=openai))

---

##  Research Frontiers and Emerging Paradigms

- **Semantically-Centric Frameworks for Multimedia AI**  
  A new research framework proposes integrating semantic information theory—introducing metrics like semantic entropy and mutual information—to improve the fidelity and efficiency of generative models in multimedia contexts.  
  ([arxiv.org](https://arxiv.org/abs/2508.17163?utm_source=openai))

- **Evolutionary Computation as Natural Generative AI (NatGenAI)**  
  A recent study repositions evolutionary computation within the generative AI paradigm, arguing it enables more creative, out-of-distribution generation via exploratory search, disrupting conventional gradient-based approaches.  
  ([arxiv.org](https://arxiv.org/abs/2510.08590?utm_source=openai))

- **Quantum Generative Models Achieving Beyond-Classical Advantage**  
  Researchers showcased quantum generative models on a 68-qubit superconducting processor that can learn and sample distributions inaccessible to classical computing—demonstrating both training and generation in beyond-classical regimes with no barren plateaus.  
  ([arxiv.org](https://arxiv.org/abs/2509.09033?utm_source=openai))

---

## Summary Table of Key Trends (Nov 2025)

- Model Innovation: GPT‑5, Gemini 2.5 Pro, Nano Banana, Qwen3, Llama 4, Runway Gen‑4  
- Hardware & Infrastructure: Qualcomm accelerators, Nvidia Blackwell/Rubin chips, OpenAI‑Oracle Stargate, AWS GAIA  
- Creative & Media: TIME AI Agent, AI‑enhanced film production  
- Research & Theory: Semantic multimedia frameworks, NatGenAI, quantum generative advantage  

---

### Final Takeaways

1. **Diversification of Generative AI**: We see parallel innovation across text (GPT‑5), image (Nano Banana), video (Gen‑4, Sora), and even robotics/world modeling (Nvidia, DeepMind).  
2. **Speed and Reasoning Take Center Stage**: Models increasingly blend fast throughput with deep reasoning (GPT‑5, Gemini 2.5 Pro, Qwen3).  
3. **Infrastructure Scaling**: Massive computation investments—from data centers to specialized chips—are accelerating AI's scalability and efficiency.  
4. **Creative Synergy**: Media brands like TIME and filmmakers like Aronofsky are embedding generative AI in storytelling and journalism.  
5. **New Paradigms Emerge**: Evolutionary and quantum generative models, along with semantic frameworks, are pushing the boundaries of what "generation" means in AI.

---

Let me know if you'd like to dive deeper into any specific topic—model architecture, hardware ecosystems, creative use cases, or theoretical advances—and I can provide more detailed insights.

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [16]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Here is a structured and in‑depth overview of the **latest developments in generative AI** as of Mon

### Continue Query with Web Search

In [17]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [18]:
display(Markdown(continue_search_response.output_text))

Here are some of the **most recent and diverse news highlights** related to **generative AI**, curated from today’s publications (Monday, November 10, 2025). I’ve organized the updates by theme and included at least two citations per point for accuracy and depth.

---

Generative AI in Action  
• **Physical AI and Vision-Language-Action (VLA) Systems**  
  A new era is dawning where AI merges cognition with movement. Vision-language-action (VLA) systems now allow robots to interpret natural language instructions, understand their environments, and execute tasks autonomously—signaling a major leap in embodied generative AI capabilities. ([telecomreview.com](https://telecomreview.com/articles/reports-and-coverage/27220-ai-steps-into-the-real-world-analyzing-the-rise-of-physical-ai/?utm_source=openai))  

• **Generative AI as a Business Risk**  
  As businesses increasingly adopt generative technologies—from customer service to product development—they face growing security threats. Experts warn that these powerful AI tools may expose organizations to unprecedented vulnerabilities if not properly managed. ([webpronews.com](https://www.webpronews.com/genais-hidden-perils-fortifying-business-defenses-in-2025/?utm_source=openai))  

Infrastructure & Ecosystem  
• **Google Cloud Introduces Ironwood TPUs and Arm-Based Axion VMs**  
  Google Cloud today launched its 7th-generation Ironwood TPUs, offering up to a 10× performance gain over TPU v5p and 4× better performance per chip than TPU v6e. Paired with new cost-efficient Arm-based Axion VMs—like the N4A instance that delivers twice the price-performance of x86—the offerings slash latency and serving costs, reshaping how AI workloads are scaled and deployed. ([quantumzeitgeist.com](https://quantumzeitgeist.com/google-cloud-ai-inference/?utm_source=openai))  

Market & Industry Trends  
• **India’s Generative AI Market Set for Explosive Growth**  
  A new report by IMARC Group reveals that India’s generative AI market grew to USD 1.30 billion in 2024 and is expected to soar to USD 5.40 billion by 2033. A 15.2% CAGR is projected through 2025–2033, driven by adoption in media, advertising, healthcare, and personalized content creation. ([openpr.com](https://www.openpr.com/news/4261720/india-generative-ai-market-size-share-trends-growth-report?utm_source=openai))  

• **Surge in Semiconductor Investment: The “Silicon Supercycle”**  
  The booming demand for high-performance computing to power AI is fueling a dramatic upswing in semiconductor investments. Market sentiment is optimistic, and analysts suggest the industry may be on track toward a trillion-dollar valuation by decade’s end. ([markets.financialcontent.com](https://markets.financialcontent.com/wral/article/tokenring-2025-11-10-the-ai-gold-rush-semiconductor-investments-soar-amidst-global-tech-transformation?utm_source=openai))  
  However, this rapid growth also brings volatility: valuations are rising, but risks are increasingly inherent in the AI-driven chip market. ([markets.financialcontent.com](https://markets.financialcontent.com/wral/article/tokenring-2025-11-10-semiconductor-stocks-navigate-ai-boom-a-volatile-ascent-amidst-trillion-dollar-dreams?utm_source=openai))  

Global Engagement & Policy  
• **GBA AI and Robotics Summit Launches in Hong Kong**  
  The “GBA International Artificial Intelligence and Robotics Summit 2025,” hosted by the Hong Kong Productivity Council, officially opened on November 10. Under the theme “Empowering Resilient Industries through Embodied AI,” the summit convenes experts from across Hong Kong, Mainland China, and several global partners to explore industrial-scale adoption of embodied AI. ([laotiantimes.com](https://laotiantimes.com/2025/11/10/gba-international-artificial-intelligence-and-robotics-summit-2025-opens-grandly-ai-and-robotics-fest-launches-alongside-hkpc-drives-ai-for-all-and-embodied-ai-adoption-to-propel-hong-kong-into-a-ne/?utm_source=openai))  

---

Overall, these developments showcase a multifaceted trajectory in generative AI:

- From **robotic embodiment** and **AI security concerns**
- To **hardware breakthroughs** through custom accelerators like Ironwood and Axion
- To **market expansion** in countries such as India
- To **investment surges** underpinning a booming semiconductor ecosystem
- And **international policy and collaboration**, notably in Hong Kong

Let me know if you'd like to explore any of these topics further—such as technical specifications of Ironwood TPUs, details of VLA systems, or breakdowns of market forecasts in India.

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [19]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [20]:
display(Markdown(combined_search_response.output_text))

Here’s a comprehensive and up-to-date overview of the **latest developments in generative AI** as of November 10, 2025. This analysis draws on recent news, model releases, market trends, and emerging applications.

---

##  Major Model Releases and Innovations

- **OpenAI GPT‑5**  
  Released on **August 7, 2025**, GPT‑5 is a multimodal foundation model that integrates reasoning and non-reasoning capabilities under a unified interface. It is accessible via ChatGPT, Microsoft Copilot, and the OpenAI API, and represents a significant leap in performance across benchmarks. ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai))

- **OpenAI o4‑mini**  
  Launched on **April 16, 2025**, this lightweight multimodal model supports both text and image inputs, including whiteboard sketch analysis and chain-of-thought reasoning. A higher-accuracy variant, o4‑mini‑high, is available to paid-tier users. ([en.wikipedia.org](https://en.wikipedia.org/wiki/OpenAI_o4-mini?utm_source=openai))

- **OpenAI GPT‑4.1**  
  Released on **April 14, 2025**, GPT‑4.1 (along with mini and nano variants) offers improved coding capabilities and is available to ChatGPT Plus and Pro subscribers. ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-4.1?utm_source=openai))

- **Google DeepMind’s Gemini 2.5 Family**  
  Google’s Gemini 2.5 Pro and Flash models, featuring enhanced reasoning, coding, and “Deep Think” capabilities, became generally available on **June 17, 2025**. A Flash‑Lite variant optimized for speed and cost-efficiency was also introduced. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gemini_%28language_model%29?utm_source=openai))

- **Nano Banana (Gemini 2.5 Flash Image)**  
  Released on **August 26, 2025**, this image generation and editing model went viral for its photorealistic “3D figurine” outputs. It supports subject consistency, multi-image fusion, and SynthID watermarking, and quickly amassed over 10 million new users and 200 million image edits. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai))

- **Google Gemini Diffusion**  
  An experimental model that applies diffusion techniques to text generation, enabling simultaneous token generation and mid-process error correction. It achieves speeds up to 1,479 tokens per second—far exceeding Gemini 2.5 Flash (~400 tps) and GPT‑4o (~150 tps). ([spglobal.com](https://www.spglobal.com/market-intelligence/en/news-insights/research/generative-ai-digest-a-wave-of-notable-ai-model-launches?utm_source=openai))

- **Alibaba Qwen3 Series**  
  Released in April 2025, Qwen3 includes open-source models ranging from 0.6B to 32B parameters, plus Mixture-of-Experts variants. They support “thinking” and “non-thinking” modes and perform strongly on coding, math, and instruction-following tasks. ([spglobal.com](https://www.spglobal.com/market-intelligence/en/news-insights/research/generative-ai-digest-a-wave-of-notable-ai-model-launches?utm_source=openai))

- **Anthropic Claude Haiku 4.5**  
  A compact model delivering flagship-level reasoning and coding performance at 4–5× the speed and roughly one-third the cost of larger models. It features a 200K token context window and agentic capabilities like “using computers” to automate tasks. ([voxfor.com](https://www.voxfor.com/what-is-new-in-ai-the-latest-news-from-october-2025/?utm_source=openai))

---

##  Emerging Applications & Ecosystem Integration

- **TIME AI Agent**  
  Launched **today**, this platform integrates language understanding, voice synthesis, translation, and search to deliver interactive, personalized journalism. Developed with Scale AI, it emphasizes editorial accuracy and transparency. ([time.com](https://time.com/7332572/the-story-behind-the-time-ai-agent/?utm_source=openai))

- **Microsoft MAI‑Image‑1**  
  Released **5 days ago**, this proprietary text-to-image generator offers fast, photorealistic image creation via Bing Image Creator and Copilot Audio Expressions. It complements OpenAI models and reflects Microsoft’s push for in-house AI capabilities. ([windowscentral.com](https://www.windowscentral.com/artificial-intelligence/microsoft-copilot/microsoft-launches-mai-image-1?utm_source=openai))

- **Google DS STAR**  
  A multi-agent framework unveiled recently that translates ambiguous business problems into executable Python code, handling mixed-format data (CSV, JSON, Markdown, unstructured text) without human intervention. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

- **Meta Vibes Expansion**  
  Meta’s AI-generated short video platform, Vibes, has expanded into Europe, offering TikTok-like experiences where every video is AI-generated. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

- **Google Maps + Gemini Integration**  
  Gemini AI is being integrated into Google Maps as a voice assistant to simplify navigation via natural language. The rollout is global and ongoing. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

- **HeyGen AI Video Translator**  
  This tool delivers hyper-realistic localization by matching tone, expressions, and lip movements in translated videos. Available via web, iOS, and API with free trial credits. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

- **ClickUp 4.0**  
  Released recently, this update introduces AI agents and a redesigned UI, integrating task management, collaboration, messaging, scheduling, and enterprise search into a unified platform. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

- **OpenAI Sora for Android**  
  Launched on **November 6, 2025**, Sora is a video app that achieved nearly 470,000 downloads on its first day—surpassing its iOS debut by 327%. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

##  Market Trends & Strategic Insights

- **Generative AI Market Growth**  
  The generative AI market experienced triple-digit growth across hardware, foundation models, and development platforms in 2024. Cloud providers are investing heavily in data centers, with over **US$400 billion** expected in AI-related spending in 2025. ([businesswire.com](https://www.businesswire.com/news/home/20250825682581/en/Generative-AI-Market-Report-2025-GenAI-Market-Experienced-Triple-digit-growth-Rates-in-All-Three-Major-Segments-Spanning-GenAI-Hardware-Foundation-Models-and-Development-Platforms---ResearchAndMarkets.com?utm_source=openai))

- **Enterprise Adoption & Scaling**  
  According to the 2025 McKinsey Global Survey, about one-third of organizations are scaling AI programs, while 39% are experimenting with agentic AI systems. High performers are more likely to redesign workflows, define human validation processes, and allocate over 20% of digital budgets to AI. ([mckinsey.com](https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai?utm_source=openai))

- **Responsible AI Practices**  
  UC Berkeley’s Responsible Use of Generative AI playbook outlines 10 actionable strategies—five for business leaders and five for product managers—to ensure ethical and responsible GenAI deployment. ([weforum.org](https://www.weforum.org/stories/2025/06/responsible-generative-ai-product-development-use/?utm_source=openai))

- **Productivity Impact Study**  
  A randomized controlled trial found that experienced open-source developers using early-2025 AI tools took **19% longer** to complete tasks, suggesting that AI may introduce friction in complex workflows. ([metr.org](https://metr.org/blog/2025-07-10-early-2025-ai-experienced-os-dev-study/?utm_source=openai))

---

##  Summary

The generative AI landscape in late 2025 is defined by:

- **Advanced multimodal models** like GPT‑5, Gemini 2.5, and Claude Haiku 4.5 pushing the boundaries of reasoning, context, and efficiency.
- **Rapid integration** of AI into everyday tools—from journalism (TIME AI Agent) to navigation (Google Maps) and productivity platforms (ClickUp).
- **Market momentum**, with massive investments and enterprise adoption accelerating the shift from experimentation to scaling.
- **Ethical and practical challenges**, including responsible deployment and real-world productivity impacts.

Let me know if you'd like a deeper dive into any specific model, application, or trend!

# 🧩 Try It Yourself: Two-Step RAG (Private Data + Combined Search)

## Step 1 — Upload & Create Vector Store
1. Upload a short text file (e.g., `my_notes.txt`) to your notebook instance.  
2. Create a **vector store** and **ingest** your uploaded file.  
3. Run a simple test query to verify retrieval:  

In [21]:

tiy_vector_store = client.vector_stores.create(
    name="tiy_vector_store"
)
tiy_vector_store_id = tiy_vector_store.id
print("Vector store created:", tiy_vector_store_id)

with open("343Lab10NewsClip.txt", "rb") as f:
    uploaded_file = client.files.create(
        file=f,
        purpose="assistants"
    )

print("File uploaded:", uploaded_file.id)


attach_result = client.vector_stores.files.create(
    vector_store_id=tiy_vector_store_id,
    file_id=uploaded_file.id,
)
print("File attached to vector store:", attach_result.id)


test_query = "Summarize the main ideas from my notes."
search_results = client.vector_stores.search(
    vector_store_id=tiy_vector_store_id,
    query=test_query,
)

print("\nTop retrieved chunks:")
for item in search_results.data[:3]:
    print("-" * 40)
    print(item.content[0].text.strip())


Vector store created: vs_69126a90413881919bb1ac715bab912f
File uploaded: file-NNFBvy1y5VYAuKHpcsu8Yi
File attached to vector store: file-NNFBvy1y5VYAuKHpcsu8Yi

Top retrieved chunks:


## Step 2 — Combine File Search with Web Search
1. Enable both **file_search** and **web_search** in the Responses API.  
2. Use a prompt that asks the model to merge insights from both sources.  
   > Example: “Using my uploaded notes and the latest web information, summarize the current trends on this topic.”  
3. Review how the answer from your file and **current info** from the web.

✅ You’ve created a RAG system that combines **private** and **public** data for comprehensive, up-to-date analysis.


In [26]:

query = (
    "Using the uploaded news article as the primary source, and also checking the latest web information, "
    "give me a short, structured summary of the topic (El Paso Texas and change following immigration crackdown). Clearly separate 'From my file' vs 'From the web'."
)

combined_response = client.responses.create(
    model="gpt-4o",
    input=query,
    temperature=0,
    instructions=(
        "First look up relevant passages from the attached vector store. "
        "Then augment with web_search to bring in current/public info. "
        "Present the answer in two sections: 'From my file(s)' and 'From the web'."
    ),
    tools=[
        {
            "type": "file_search",
            "vector_store_ids": [tiy_vector_store_id],
        },
        {
            "type": "web_search"
        }
    ],
)

display(Markdown(combined_response.output_text))


Here is a structured summary of the topic **El Paso, Texas, and changes following the immigration crackdown**, divided into two sections as requested:

From my file(s)  
----------------  
*(No relevant content was found in the uploaded file(s) regarding El Paso, Texas, and changes following an immigration crackdown. Please let me know if you'd like me to search again or if there’s a specific document I should focus on.)*

From the web  
------------

**1. Expansion of Military Enforcement**  
- In May 2025, the U.S. Department of Defense designated a second military zone along the U.S.–Mexico border, extending into Texas and connecting with Fort Bliss in El Paso. This allows military personnel to detain migrants who cross illegally until they are transferred to DHS, bypassing the Posse Comitatus Act.([apnews.com](https://apnews.com/article/d7d15f23bd755b95cd90cbb9a89df6fa?utm_source=openai))  
- Under Operation Lone Star, Texas has deployed National Guard troops to El Paso. In early 2025, an agreement with the Trump administration authorized Guard members to make immigration arrests under CBP supervision.([elpasomatters.org](https://elpasomatters.org/2025/02/03/texas-national-guard-immigration-arrests-under-trump-abbott-agreement/?utm_source=openai))

**2. Detention Infrastructure Expansion**  
- In August 2025, ICE opened the “Lone Star Lockup,” a mega detention facility at Fort Bliss in El Paso. Initially holding 1,000 people, it is planned to expand to 5,000 beds with a $1 billion investment. The facility has drawn criticism over transparency, access to legal counsel, and human rights concerns.([time.com](https://time.com/7310657/ice-immigration-detention-texas-lone-star-lockup/?utm_source=openai))

**3. Humanitarian and Community Impact**  
- Faith leaders in El Paso, including Catholic Bishop Mark Seitz, have raised alarms over the crackdown’s impact on migrants and humanitarian organizations. Policies extending enforcement into “sensitive locations” like churches and schools have instilled fear, disrupted legal aid services, and led some migrants to return to dangerous conditions in their home countries.([kwbu.org](https://www.kwbu.org/news-from-across-texas/2025-06-27/fear-and-faith-el-paso-church-leaders-sound-the-alarm-amid-ice-crackdown?utm_source=openai))  
- Legal aid organizations such as Estrella del Paso have lost significant federal funding—about $5 million—and had to furlough staff. Emergency grants have provided temporary relief.([kwbu.org](https://www.kwbu.org/news-from-across-texas/2025-06-27/fear-and-faith-el-paso-church-leaders-sound-the-alarm-amid-ice-crackdown?utm_source=openai))

**4. Legal and Political Developments**  
- The Texas Supreme Court ruled in May 2025 that Attorney General Ken Paxton may resume his investigation into Annunciation House, a Catholic migrant shelter in El Paso, potentially revoking its charter.([houstonchronicle.com](https://www.houstonchronicle.com/politics/texas/article/annunciation-house-ken-paxton-20353327.php?utm_source=openai))  
- In federal court, the Fifth Circuit upheld an injunction blocking Texas’s S.B. 4, a state law allowing local officials to arrest and deport migrants, reaffirming federal authority over immigration.([en.wikipedia.org](https://en.wikipedia.org/wiki/United_States_v._Texas_%282024%29?utm_source=openai))

**5. Rising Migrant Deaths and Shifting Routes**  
- Since El Paso joined Operation Lone Star in late 2022, the number of migrant deaths in the region has surged. From January 2023 to August 2024, 299 human remains were found in the El Paso sector—more than double the 122 recorded in the prior 20 months.([click2houston.com](https://www.click2houston.com/news/texas/2025/06/16/after-el-paso-joined-abbotts-border-crackdown-the-number-of-dead-migrants-in-the-new-mexico-desert-surged/?utm_source=openai))  
- The militarized enforcement has pushed migrants into more dangerous routes, particularly through the New Mexico desert, where deaths have increased significantly.([texastribune.org](https://www.texastribune.org/2025/06/16/texas-operation-lone-star-border-el-paso-deaths-migrants-new-mexico/?utm_source=openai))

**Summary**  
El Paso has become a focal point of intensified immigration enforcement, marked by military zone designations, expanded detention capacity, and state-federal cooperation in arrests. These measures have strained humanitarian services, provoked legal battles, and contributed to a tragic rise in migrant deaths. Community leaders and faith-based organizations continue to resist and adapt, even as legal and political pressures mount.

Let me know if you'd like a deeper dive into any of these areas or additional context.